In [1]:
import requests
import json
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import os
from getpass import getpass
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
import time
from pymongo import MongoClient
from dotenv import dotenv_values
import googlemaps

##### **I am going to look at other stores of the same franchise and take the top 20 in visits from Bogota, Medellin and Cali, then I will take the 4 best of each city and find out what they have in common.**

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("google_key") 

In [4]:
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
bogota = "Interrapidisimo Bogotá"

In [8]:

params = {
    "query": bogota,
    "key": api_key,
    "region": "co",
    "language": "es",
    "maxResults": 20
}

response = requests.get(url, params=params)
data = json.loads(response.text)

results = []

# Loop through the data and extract the necessary information
for result in data["results"]:
    name = result["name"]
    lat = result["geometry"]["location"]["lat"]
    lng = result["geometry"]["location"]["lng"]
    num_reviews = result.get("user_ratings_total", 0)
    address = result.get("formatted_address", "")
    rating = result.get("rating", 0)

    results.append({
        "Name": name,
        "Latitude": lat,
        "Longitude": lng,
        "Number of Reviews": num_reviews,
        "Address": address,
        "Rating": rating
    })


In [9]:
df_bogota = pd.DataFrame(results)

In [10]:
df_bogota.sort_values(by="Number of Reviews", ascending=False)

,Name,Latitude,Longitude,Number of Reviews,Address,Rating
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá",3.7
14,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá",4.1
5,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá",4.2
19,Inter Rapidisimo,4.668435,-74.142324,154,"Cl. 17 #96c-26, Bogotá",3.8
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá",4.3
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca",4.3
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá",4.3
15,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca",4.3
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá",3.8
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca",4.0


In [11]:
import requests

# Define la URL de la API Geocoding de Google Maps
geocoding_url = "https://maps.googleapis.com/maps/api/geocode/json"


# Define una función para obtener el Place ID de una ubicación a partir de sus coordenadas de latitud y longitud
def get_place_id(lat, lng):
    # Define los parámetros de la solicitud de geocodificación inversa
    params = {
        "latlng": f"{lat},{lng}",
        "key": api_key,
    }
    # Realiza la solicitud a la API Geocoding de Google Maps
    response = requests.get(geocoding_url, params=params)
    # Convierte la respuesta a un diccionario de Python
    response_dict = response.json()
    # Extrae el Place ID de la respuesta
    place_id = response_dict["results"][0]["place_id"]
    # Retorna el Place ID
    return place_id

# Añade una nueva columna al dataframe con el Place ID de cada ubicación
df_bogota["Place ID"] = df_bogota.apply(lambda row: get_place_id(row["Latitude"], row["Longitude"]), axis=1)


In [12]:
df_bogota = df_bogota.sort_values(by="Number of Reviews", ascending=False)
df_bogota

,Name,Latitude,Longitude,Number of Reviews,Address,Rating,Place ID
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá",3.7,ChIJX9S2LmuZP44RFrAJBi-_ucg
14,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá",4.1,ChIJEQasPHqFP44RuNaHnt3lub0
5,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá",4.2,ChIJSUDyT0OFP44R3f2qlUkhN0Q
19,Inter Rapidisimo,4.668435,-74.142324,154,"Cl. 17 #96c-26, Bogotá",3.8,ChIJIy_2B4qcP44Rm0XffYtBijk
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá",4.3,ChIJe_PYGRGcP44RQ_66Seam4CY
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca",4.3,ChIJ-4-3FyKcP44R6OzgcEmPSIQ
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá",4.3,ChIJFeuBdSeZP44RJVxTw19phCg
15,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca",4.3,ChIJbTI7GIePP44RaLCGRODQma8
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá",3.8,ChIJaYVbTtuZP44RRukjkIexULM
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca",4.0,ChIJB4jkVmCFP44R6NAZ9Ns77e8


In [13]:
# Crea un mapa centrado en Bogotá
map_bogota = folium.Map(location=[4.60971, -74.08175], zoom_start=12)

# Itera sobre cada fila de tu dataframe
for index, row in df_bogota.iterrows():
    # Obtén la latitud y longitud de la ubicación actual
    lat = row['Latitude']
    lng = row['Longitude']
    # Agrega un marcador en el mapa para la ubicación actual
    popup_text = f"{row['Name']}<br>Number of Reviews: {row['Number of Reviews']}"
    if row['Number of Reviews'] >= 80:
        marker_color = 'red'
    else:
        marker_color = 'blue'
    folium.Marker(location=[lat, lng], popup=popup_text, icon=folium.Icon(color=marker_color)).add_to(map_bogota)

# Visualiza el mapa
map_bogota

In [33]:
df_bogota["Address"]

0                              Ave Cra 30 #7-45, Bogotá
13                              Cl. 161 #16B-48, Bogotá
17                              Cra. 16 #136-91, Bogotá
10                                 Cl. 9 #77-31, Bogotá
12        Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca
3                                 Cra 27 #22-42, Bogotá
14                         110131, Bogotá, Cundinamarca
1                         CRA 86 # 34 B -14 SUR, Bogotá
6       Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca
11                     DgG, Dg. 52 Sur #54 A 40, Bogotá
15                               Cl. 150 #48-11, Bogotá
18                            Cra. 81a #13 D 39, Bogotá
19                              Cra. 82 #23A 25, Bogotá
7     Cra. 13 #57-28, Localidad de Chapinero, Bogotá...
2        Av. 1 de Mayo #51F - 47, Puente Aranda, Bogotá
9                 Cra. 28a #17-14, Los Mártires, Bogotá
5                               Cl. 79 #42 # 18, Bogotá
16       Cl. 72 #100 22, Engativá, Bogotá, Cundi

In [14]:
estratos = [3, 2, 4, 3, 2, 2, 3, 2, 5, 2, 5, 4, 4, 5, 3, 3, 5, 3, 3, 4]
df_bogota["Estratos"] = estratos


In [15]:
df_bogota

,Name,Latitude,Longitude,Number of Reviews,Address,Rating,Place ID,Estratos
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá",3.7,ChIJX9S2LmuZP44RFrAJBi-_ucg,3
14,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá",4.1,ChIJEQasPHqFP44RuNaHnt3lub0,2
5,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá",4.2,ChIJSUDyT0OFP44R3f2qlUkhN0Q,4
19,Inter Rapidisimo,4.668435,-74.142324,154,"Cl. 17 #96c-26, Bogotá",3.8,ChIJIy_2B4qcP44Rm0XffYtBijk,3
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá",4.3,ChIJe_PYGRGcP44RQ_66Seam4CY,2
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca",4.3,ChIJ-4-3FyKcP44R6OzgcEmPSIQ,2
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá",4.3,ChIJFeuBdSeZP44RJVxTw19phCg,3
15,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca",4.3,ChIJbTI7GIePP44RaLCGRODQma8,2
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá",3.8,ChIJaYVbTtuZP44RRukjkIexULM,5
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca",4.0,ChIJB4jkVmCFP44R6NAZ9Ns77e8,2


In [24]:
def calculate_average_income(df):
    # Define the exchange rate (in Colombian pesos per US dollar)
    exchange_rate = 5000
    
    # Calculate the average income by estrato
    avg_income_by_estrato = {
        1: 907435 / exchange_rate,
        2: 1434438 / exchange_rate,
        3: 2396912 / exchange_rate,
        4: 3926246 / exchange_rate,
        5: 4666754 / exchange_rate,
        6: 6476799 / exchange_rate
    }
    
    # Create a new column "average_inc" with the average income by estrato
    df["average_inc_us"] = df["Estratos"].map(avg_income_by_estrato)
    
    return df

# Apply the function to the data
df_bogota = calculate_average_income(df_bogota)


In [25]:
df_bogota

,Name,Latitude,Longitude,Number of Reviews,Address,Rating,Place ID,Estratos,average_inc,average_inc_us
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá",3.7,ChIJX9S2LmuZP44RFrAJBi-_ucg,3,479.3824,479.3824
14,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá",4.1,ChIJEQasPHqFP44RuNaHnt3lub0,2,286.8876,286.8876
5,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá",4.2,ChIJSUDyT0OFP44R3f2qlUkhN0Q,4,785.2492,785.2492
19,Inter Rapidisimo,4.668435,-74.142324,154,"Cl. 17 #96c-26, Bogotá",3.8,ChIJIy_2B4qcP44Rm0XffYtBijk,3,479.3824,479.3824
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá",4.3,ChIJe_PYGRGcP44RQ_66Seam4CY,2,286.8876,286.8876
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca",4.3,ChIJ-4-3FyKcP44R6OzgcEmPSIQ,2,286.8876,286.8876
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá",4.3,ChIJFeuBdSeZP44RJVxTw19phCg,3,479.3824,479.3824
15,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca",4.3,ChIJbTI7GIePP44RaLCGRODQma8,2,286.8876,286.8876
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá",3.8,ChIJaYVbTtuZP44RRukjkIexULM,5,933.3508,933.3508
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca",4.0,ChIJB4jkVmCFP44R6NAZ9Ns77e8,2,286.8876,286.8876


In [47]:
def get_types(lat, lng):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lng}",
        "radius": 200,
        "key": api_key,
        "type": "establishment",
        "language": "es"
    }
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    types = []
    for result in data["results"]:
        for t in result.get("types", []):
            if t not in types and t != "locality" and t != "political" and t !="establishment":
                types.append(t)
            if len(types) == 10:
                break
        if len(types) == 10:
            break
    return types

df_bogota["Types"] = df_bogota.apply(lambda row: get_types(row["Latitude"], row["Longitude"]), axis=1)



In [54]:
df_bogota

,Name,Latitude,Longitude,number_reviews,Address,Rating,Estratos,average_inc_us,type_of_places
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá",3.7,3,479.3824,"[gas_station, point_of_interest, moving_compan..."
14,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá",4.1,2,286.8876,"[supermarket, grocery_or_supermarket, food, po..."
5,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá",4.2,4,785.2492,"[bank, finance, point_of_interest, church, pla..."
19,Inter Rapidisimo,4.668435,-74.142324,154,"Cl. 17 #96c-26, Bogotá",3.8,3,479.3824,"[point_of_interest, finance, car_repair, store..."
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá",4.3,2,286.8876,"[store, point_of_interest, locksmith, clothing..."
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca",4.3,2,286.8876,"[church, place_of_worship, point_of_interest, ..."
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá",4.3,3,479.3824,"[point_of_interest, general_contractor, store,..."
15,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca",4.3,2,286.8876,"[insurance_agency, finance, school, store, poi..."
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá",3.8,5,933.3508,"[post_office, finance, point_of_interest, stor..."
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca",4.0,2,286.8876,"[store, point_of_interest, doctor, health, ele..."


In [50]:
df_bogota.drop("Place ID", axis=1, inplace=True)


In [52]:
df_bogota.drop("average_inc", axis=1, inplace=True)

In [53]:
df_bogota.rename(columns={"Number of Reviews": "number_reviews","Types": "type_of_places" }, inplace=True)
